In [458]:
#package
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
# from pyarrow import csv
from datetime import datetime 
import numpy as np
import os, random, math, time, shutil
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Modeling
from fbprophet import Prophet

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else: 
    rc('font', family='AppleGothic')

# 모델링
import sys
import os
sys.path.append(os.path.join(os.getcwd(),'module'))

from rfts.basemodel import basemodel_timeseries
from rfts.basemodel import basemodel_timeseries_decomp
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
from catboost import CatBoostRegressor

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import datetime
# from pyarrow import csv
import pickle
import re

# error 함수
def MAPE(y_test, y_pred): return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 
from sklearn.metrics import mean_absolute_error as mae

In [459]:
#data
data = pd.read_csv('data/final_merged_by_day_v7.csv')
data["base_date"] = pd.to_datetime(data.base_date)

In [460]:
def modeling_pred(df, emd, x_var, start0 = "2018-01-01", plot=True):

    ##############################################
    add_model = Prophet(weekly_seasonality=True, daily_seasonality=False, yearly_seasonality=True, 
                        changepoint_prior_scale=0.05)  
    model_season = 0
    multi_model = Prophet(weekly_seasonality=True, daily_seasonality=False, yearly_seasonality=True, 
                        changepoint_prior_scale=0.05, seasonality_mode='multiplicative')
    train_m = df[df["emd_nm"]==emd]; perio_m = 1
    tmp_m = train_m[["base_date", x_var]] ; tmp_m.index = tmp_m.base_date; tmp_m.columns = ["ds", "y"]
    add_model.fit(tmp_m) ; m2 = multi_model.fit(tmp_m)
    future_m = add_model.make_future_dataframe(periods=perio_m)
    add_forecast = add_model.predict(future_m) ; multi_forecast = multi_model.predict(future_m)
    add_error = mae(tmp_m.y, add_forecast[['yhat']].head(len(tmp_m)))
    multi_error = mae(tmp_m.y, multi_forecast[['yhat']].head(len(tmp_m)))
    if add_error > multi_error:
        model_season = 1
    ##############################################
    
    tmp = df.loc[(df.emd_nm==emd) & (df.base_date>=start0)]
    tmp.index = tmp.base_date
    tmp = tmp.drop(["emd_nm"], axis=1)
    tmp.columns = ["ds", "y"]
    
    if model_season==1:
            model = Prophet(weekly_seasonality=True, daily_seasonality=False, yearly_seasonality=True, 
                            changepoint_prior_scale=0.05, seasonality_mode='multiplicative')
    else: model = Prophet(weekly_seasonality=True, daily_seasonality=False, yearly_seasonality=True, 
                                changepoint_prior_scale=0.05)  
        
    model.fit(tmp)
    future = model.make_future_dataframe(periods=62)
    forecast = model.predict(future)
    
    sum_7 = sum(forecast.yhat[len(tmp):len(tmp)+31])
    sum_8 = sum(forecast.yhat[len(tmp)+31: ])
    forc = forecast[["ds","yhat"]]

    print("{} 7월 예측값: ".format(emd) , sum_7 )
    print("{} 8월 예측값: ".format(emd) , sum_8 )
    return sum_7,sum_8, forc

In [461]:
# 지역 그룹핑
emd_dict = { "jeju_1": ['애월읍'], 
             "jeju_2": ['구좌읍', '조천읍', '한경면', '한림읍'],
             "jeju_3": ['일도1동','노형동','이도2동', '연동','일도2동',
                        '아라동','삼양동','외도동','화북동','삼도1동','용담2동','오라동',
                        '이도1동','건입동','삼도2동','용담1동',
                        '이호동','봉개동','도두동'],
             "seo_1": ["대정읍","안덕면","성산읍","표선면"], 
             "seo_2": ["영천동","효돈동"], 
             "seo_3": ['송산동','천지동','정방동','동홍동','서홍동',
                      '예래동','남원읍','대륜동','중앙동','중문동','대천동'],
             "알수없음": ['알수없음']}

In [462]:
#prophet
df = data[["base_date", "em_g", "emd_nm"]]
start = time.time()  # 시작 시간 저장
x_var = "em_g" 
output = pd.DataFrame()
prophet_result =  pd.DataFrame()
for emd in tqdm(data.emd_nm.unique()):
    print("================{}=================".format(emd))
    result_= []
    if emd in emd_dict.get("seo_1"):start_date = "2020-02-01"
    elif emd in emd_dict.get("jeju_1")+emd_dict.get("jeju_2"):start_date = "2020-01-01"
    else: start_date = "2018-01-01"
    pred7, pred8, forc_ = modeling_pred(df,emd,x_var,start0=start_date)
    result_.append(pred7); result_.append(pred8);
    res_df = pd.DataFrame(result_).T
    res_df.columns = ["pred_7", "pred_8"]
    res_df["emd_nm"] = emd
    output = pd.concat([output,res_df],axis=0)
    forc_["emd_nm"]=emd
    prophet_result = pd.concat([prophet_result,forc_], axis=0)
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

  0%|          | 0/41 [00:00<?, ?it/s]

================남원읍=================
Initial log joint probability = -13.5172
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2205.27    0.00272323       95.1637      0.1432           1      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     171       2209.37   0.000346571       75.1688   7.029e-06       0.001      280  LS failed, Hessian reset 
     199       2209.89   0.000244397       76.8856      0.6448      0.6448      313   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     243       2210.22    0.00137982       62.1712    1.52e-05       0.001      400  LS failed, Hessian reset 
     286       2210.31   1.54953e-07       77.7372      0.2982           1      457   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
Initial log joint probability = -13.5172
    Iter      lo

  2%|▏         | 1/41 [00:04<03:04,  4.62s/it]

남원읍 7월 예측값:  71915614.5934119
남원읍 8월 예측값:  73437275.75656362
================대륜동=================
Initial log joint probability = -8.87841
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2686.96   0.000393788       137.658      0.5589      0.5589      113   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     158       2694.02    0.00144141       197.998   1.455e-05       0.001      218  LS failed, Hessian reset 
     199       2696.34   0.000175614       57.8613      0.7048      0.7048      270   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     212       2696.76    0.00160606       65.2439   1.298e-05       0.001      323  LS failed, Hessian reset 
     241       2697.21   0.000181825       72.2396    1.47e-06       0.001      398  LS failed, Hessian reset 
     267       2697.35   0.000174683       69.9293   3.033e-06       0.001    

  5%|▍         | 2/41 [00:09<03:07,  4.81s/it]

...

구좌읍 7월 예측값:  48607863.20581173
구좌읍 8월 예측값:  53159099.47661382
================한림읍=================
Initial log joint probability = -17.4097
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1129.19      0.017711        137.77      0.4036           1      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199        1145.6    0.00401789       104.066      0.5075           1      263   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     291       1145.89   4.07736e-08       77.7013      0.4241           1      379   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
Initial log joint probability = -17.4097
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1132.41     0.0024086       112.543      0.6555      0.6555      123 

 98%|█████████▊| 40/41 [03:03<00:03,  3.01s/it]

한림읍 7월 예측값:  95962958.08822018
한림읍 8월 예측값:  101945318.26521775
================한경면=================
Initial log joint probability = -13.1698
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1105.15     0.0173037       94.7982       6.827      0.6827      134   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     181       1107.46    0.00263653       107.823   2.467e-05       0.001      289  LS failed, Hessian reset 
     199       1107.59    4.5951e-05       66.7167      0.2337      0.9788      312   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     270       1107.59   3.14282e-05       86.1583   4.647e-07       0.001      456  LS failed, Hessian reset 
     299        1107.6   2.93494e-07       70.8223           1           1      499   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     331

100%|██████████| 41/41 [03:06<00:00,  4.55s/it]

한경면 7월 예측값:  32451132.582791246
한경면 8월 예측값:  36655877.57799886
time : 186.51748704910278


In [463]:
#prophet_result
prophet_result = prophet_result.groupby(['emd_nm',prophet_result.ds.dt.year,prophet_result.ds.dt.month]).sum()
prophet_result.index.names = ['emd_nm','year','month']
prophet_result = prophet_result.reset_index()
prophet_result = prophet_result[(prophet_result['year']==2021)&((prophet_result['month']==7)|(prophet_result['month']==8))]
prophet_result = prophet_result.drop('year',axis=1)
result = prophet_result.pivot_table(index=['emd_nm'],columns='month',values='yhat')
result.columns = ['prophet_7','prophet_8']

In [464]:
# 1. 모델 불러오기
lasso = Lasso(random_state=2020)   
elastic = ElasticNet(random_state=2020)
ridge = Ridge(random_state=2020)
cb = CatBoostRegressor(random_state=2020,verbose=False)


regr = RandomForestRegressor(random_state=2020,n_jobs=-1)

In [465]:
# train test data
train = pd.read_csv('data/final_merged_by_day_v7.csv')
train["base_date"] = pd.to_datetime(train.base_date)
train.index = pd.DatetimeIndex(train.base_date)

test = pd.read_csv('data/final_merged_by_day_v8_7_8.csv').iloc[:,1:]
test["base_date"] = pd.to_datetime(test.base_date)
test.index = pd.DatetimeIndex(test.base_date)

In [466]:
# decomposition 위해 데이터 전처리, 컬럼 정의

for i in train.columns:
    try:
        train.loc[train[i]<=0,i] = 0.001
    except:
          pass
r_data = deepcopy(train)

for i in test.columns:
    try:
        test.loc[test[i]<=0,i] = 0.001
    except:
          pass
r_data = deepcopy(test)

# column명
base_col = ['base_date', 'emd_nm', 'pay_amt', 'em_g', 'em_cnt','city','cluster']

time_col = ['holidays']

# 외국인 제거
pop_col = ['korean_resd_pop','total_resd_pop','fam', 'pop_per_fam', 'popdens']
        
pop_col_korean = ['resd_pop_cnt', 'work_pop_jeju', 'work_pop_etc', 'visit_pop_jeju','visit_pop_etc','vpc_daytm_rt']

pop_col_foreign = ['visit_short','CHN_short', 'JPN_short', 'USA_short', 'CHN_ratio_short','NotCHN_short', 
                   'resd_jeju','country_1_all_jeju','country_2_resd_jeju', 'country_2_visit_jeju', 'country_3_resd_jeju',
                   'country_4_resd_jeju', 'country_7_all_jeju', 'country_3_all_etc']

# 미세먼지, 황사 제거
weather_col = ['강수','기온','습도','풍속','태풍']

card_col = ['all_cnt','유형1_cnt', '유형1_amt',  '유형2_cnt', '유형2_amt',  '유형3_cnt',
            '유형3_amt', '농축수산물_cnt', '농축수산물_amt','배달_cnt', '배달_amt', 
            '배달_ratio','유형1_ratio','유형2_ratio','유형3_ratio', '농축수산물_ratio']

search_col = ['검색_관광', '검색_쇼핑', '검색_쓰레기', '검색_한파', '검색_강풍', '검색_태풍', '검색_장마', '검색_폭염',
               '검색_우박', '검색_날씨', '읍면동별검색_생활', '읍면동별검색_관광','읍면동별검색_쇼핑']

entrance_col = ['총계_월', '내국인전체_월', '외국인_월']

age = ['연령0020', '연령2030', '연령2040_rt','연령3040', '연령4060', '연령60이상']

In [467]:
mean_column= weather_col + entrance_col + age + pop_col

In [468]:
#train test
cl = train[['emd_nm','city','cluster']].drop_duplicates().reset_index(drop=True)
test = pd.merge(test,cl,on='emd_nm',how='left')
test.index = pd.DatetimeIndex(test.base_date)
train_test = pd.concat([train,test],axis=0)

In [469]:
print(train.shape)
print(test.shape)
print(train.base_date.min())
print(train.base_date.max())
print(test.base_date.min())
print(test.base_date.max())

(49609, 81)
(2542, 57)
2018-01-01 00:00:00
2021-06-30 00:00:00
2021-07-01 00:00:00
2021-08-31 00:00:00


In [471]:
# rf rf ylag 1 None
emd_ls_1 = ['한경면','도두동','삼도1동','삼양동','아라동','오라동','용담2동','이도2동','대정읍','효돈동','동홍동','구좌읍','화북동','영천동']
emd_ls_5 = ['한경면','건입동','노형동','도두동','삼도1동','삼양동','아라동','오라동','용담2동','이도2동','대정읍','효돈동','한림읍','동홍동','일도2동']
emd_ls_4 = ['천지동','중앙동','서홍동','삼도2동','대천동']
emd_ls_2 = ['이도1동','이호동','성산읍','안덕면','표선면','애월읍','남원읍','대륜동','송산동','예래동','정방동','중문동','천지동','구좌읍','일도1동','화북동','대천동','영천동']
emd_ls_6 = ['봉개동','연동','외도동','용담1동','이도1동','이호동','성산읍','안덕면','표선면','조천읍','남원읍','대륜동','송산동','예래동','정방동','중문동','서홍동','일도2동']
emd_ls_3 = ['일도1동','중앙동']
emd_ls = emd_ls_1 + emd_ls_5 + emd_ls_2 +emd_ls_6+emd_ls_4+emd_ls_3

In [472]:
set(emd_ls)-set(train.emd_nm.unique())

set()

In [473]:
print(set(train.emd_nm.unique())-set(emd_ls))

set()


In [474]:
def emd_parm(emd_nm):
    if emd_nm in emd_dict.get("jeju_1"):  
        return "2020-02-01",'7d',10
    elif emd_nm in emd_dict.get("jeju_2"):  
        return "2020-03-01",'7d',4
    elif (emd_nm in emd_dict.get("jeju_3")):  
        return "2018-01-01",'7d',52
    elif emd_nm in emd_dict.get("seo_1"): 
        return "2020-01-01",'7d',4
    elif emd_nm in emd_dict.get("seo_2"): 
        return "2020-01-01",'7d',4
    else:   
        return "2019-06-01",'4d',44

In [475]:
result[['7_1','7_2','7_3','7_4','7_5','7_6','8_1','8_2','8_3','8_4','8_5','8_6']] = 0

In [476]:
def func(emd_nm,col,col2):
    global result
    model = basemodel_timeseries_decomp(freq=emd_parm(emd_nm)[1],period=emd_parm(emd_nm)[2])
    train_test = pd.concat([train,test],axis=0)
    # 7월
    df = train_test[emd_parm(emd_nm)[0]:]
    if emd_nm in emd_ls_1:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=1)
        pred_res,pred_comp_res = model.predict(train_df,test_df,regr,regr)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-1]
        result.loc[emd_nm,'7_1'] = pred_res[:4].sum() + pred_res[4]*0.553 - end_value
        result.loc[emd_nm,'8_1'] = pred_res[5:].sum() + pred_res[4]*0.447
    if emd_nm in emd_ls_2:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=1)
        pred_res,pred_comp_res = model.predict(train_df,test_df,cb,cb)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-1]
        result.loc[emd_nm,'7_2'] = pred_res[:4].sum() + pred_res[4]*0.553 - end_value
        result.loc[emd_nm,'8_2'] = pred_res[5:].sum() + pred_res[4]*0.447
    if emd_nm in emd_ls_3:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=1)
        pred_res,pred_comp_res = model.predict(train_df,test_df,cb,ridge)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-1]
        result.loc[emd_nm,'7_3']= pred_res[:4].sum() + pred_res[4]*0.553 - end_value
        result.loc[emd_nm,'8_3'] = pred_res[5:].sum() + pred_res[4]*0.447
    if emd_nm in emd_ls_4:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=1)
        pred_res,pred_comp_res = model.predict(train_df,test_df,regr,ridge)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-1]
        result.loc[emd_nm,'7_4']= pred_res[:4].sum() + pred_res[4]*0.553 - end_value
        result.loc[emd_nm,'8_4'] = pred_res[5:].sum() + pred_res[4]*0.447
    if emd_nm in emd_ls_5:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=None)
        pred_res,pred_comp_res = model.predict(train_df,test_df,regr,regr)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-1]
        result.loc[emd_nm,'7_5']= pred_res[:4].sum() + pred_res[4]*0.553 - end_value
        result.loc[emd_nm,'8_5'] = pred_res[5:].sum() + pred_res[4]*0.447
    if emd_nm in emd_ls_6:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=None)
        pred_res,pred_comp_res = model.predict(train_df,test_df,cb,cb)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-1]
        result.loc[emd_nm,'7_6']= pred_res[:4].sum() + pred_res[4]*0.553 - end_value
        result.loc[emd_nm,'8_6'] = pred_res[5:].sum() + pred_res[4]*0.447

In [477]:
col = set([*search_col,'내국인전체_월','연령0020',*weather_col]) - set(['검색_폭염','검색_우박','읍면동별검색_생활'])
mean_column= weather_col + entrance_col + age + pop_col
func('애월읍',col=col,col2=mean_column)

In [479]:
col = set([*search_col,'내국인전체_월','연령0020',*weather_col]) - set(['검색_폭염','검색_우박','읍면동별검색_생활'])
mean_column= weather_col + entrance_col + age + pop_col
func('구좌읍',col=col,col2=mean_column)

In [481]:
col2 = set(col) - set(['검색_쇼핑'])
func('한림읍',col=col2,col2=mean_column)

In [482]:
col2 = set([*col,'검색_우박']) - set(['검색_쇼핑','검색_쓰레기'])
for i in ['조천읍','한경면']:
    func(i,col=col2,col2=mean_column)

In [480]:
col = set([*pop_col,*search_col,*weather_col,'내국인전체_월']) - set(['foreign_resd_pop','foreign_resd_ratio','검색_폭염','검색_우박','검색_생활','읍면동별검색_생활','읍면동별검색_쇼핑','검색_쇼핑','검색_쓰레기'])
for i in data[data['cluster']=='jeju_3']['emd_nm'].unique():
    func(i,col=col,col2=mean_column)

In [483]:
col = set([*pop_col,*search_col,*weather_col,'내국인전체_월']) - set(['foreign_resd_pop','foreign_resd_ratio','검색_폭염','검색_우박','검색_생활','읍면동별검색_생활','읍면동별검색_쇼핑','검색_쇼핑','검색_쓰레기'])
for i in ['일도2동','화북동']:
    func(i,col=col,col2=mean_column)

In [484]:
col = set([*search_col,'풍속','태풍']) - set(['검색_폭염','검색_생활'])
for i in ['대정읍','성산읍']:
    func(i,col=col,col2=mean_column)

In [485]:
col = set([*search_col,'풍속','태풍']) - set(['검색_폭염','검색_생활','읍면동별검색_쇼핑'])
func('안덕면',col=col,col2=mean_column)

In [486]:
col = set([*search_col,'풍속','태풍']) - set(['검색_강풍','검색_우박'])
func('표선면',col=col,col2=mean_column)

In [487]:
col = age + search_col + weather_col + entrance_col + pop_col
for i in ['영천동']:
    func(i,col=col,col2=mean_column)

In [488]:
col = age + search_col + weather_col + entrance_col + pop_col
for i in ['효돈동']:
    func(i,col=col,col2=mean_column)

In [489]:
def func(emd_nm,col,col2):
    global result
    model = basemodel_timeseries_decomp(freq=emd_parm(emd_nm)[1],period=emd_parm(emd_nm)[2])
    train_test = pd.concat([train,test],axis=0)
    df = train_test[emd_parm(emd_nm)[0]:]

    if emd_nm in emd_ls_5:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=1)
        pred_res,pred_comp_res = model.predict(train_df,test_df,regr,regr)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-2].sum()
        result.loc[emd_nm,'7_1'] = pred_res[:8].sum() + pred_res[8]*0.238 - end_value
        result.loc[emd_nm,'8_1'] = pred_res[9:].sum() + pred_res[8]*0.762
    if emd_nm in emd_ls_2:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=1)
        pred_res,pred_comp_res = model.predict(train_df,test_df,cb,cb)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-2].sum()
        result.loc[emd_nm,'7_2'] = pred_res[:8].sum() + pred_res[8]*0.238 - end_value
        result.loc[emd_nm,'8_2'] = pred_res[9:].sum() + pred_res[8]*0.762
    if emd_nm in emd_ls_3:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=1)
        pred_res,pred_comp_res = model.predict(train_df,test_df,cb,ridge)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-2].sum()
        result.loc[emd_nm,'7_3']= pred_res[:8].sum() + pred_res[8]*0.238 - end_value
        result.loc[emd_nm,'8_3'] = pred_res[9:].sum() + pred_res[8]*0.762
    if emd_nm in emd_ls_4:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=1)
        pred_res,pred_comp_res = model.predict(train_df,test_df,regr,ridge)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-2].sum()
        result.loc[emd_nm,'7_4']= pred_res[:8].sum() + pred_res[8]*0.238 - end_value
        result.loc[emd_nm,'8_4'] = pred_res[9:].sum() + pred_res[8]*0.762
    if emd_nm in emd_ls_5:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=None)
        pred_res,pred_comp_res = model.predict(train_df,test_df,regr,regr)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-2].sum()
        result.loc[emd_nm,'7_5']= pred_res[:8].sum() + pred_res[8]*0.238 - end_value
        result.loc[emd_nm,'8_5'] = pred_res[9:].sum() + pred_res[8]*0.762
    if emd_nm in emd_ls_6:
        train_df,test_df = model.make_df(df,emd_nm,col=col,col2=col2,split='2021-07-01',lag_col=None,x_lag=None,y_lag=None)
        pred_res,pred_comp_res = model.predict(train_df,test_df,cb,cb)
        end_value = train[train['emd_nm']==emd_nm]['em_g'][-2].sum()
        result.loc[emd_nm,'7_6']= pred_res[:8].sum() + pred_res[8]*0.238 - end_value
        result.loc[emd_nm,'8_6'] = pred_res[9:].sum() + pred_res[8]*0.762

In [490]:
col = set([*pop_col,*search_col,*weather_col,'내국인전체_월']) - set(['foreign_resd_pop','foreign_resd_ratio','검색_폭염','검색_생활','읍면동별검색_생활','읍면동별검색_쇼핑','검색_쓰레기'])

for i in data[data['cluster']=='seo_3']['emd_nm'].unique():
    func(i,col=col,col2=mean_column)

In [494]:
result.to_csv("result.csv")

In [497]:
result = result.reset_index()

In [498]:
no_prophet = ['건입동','구좌읍','남원읍','도두동','봉개동','삼도1동','성산읍','송산동','아라동','안덕면','영천동','예래동','용담1동','용담2동',
             '이도1동','일도1동','조천읍','중문동','중앙동','천지동','표선면','효돈동']

In [499]:
col7 = ['7_1', '7_2', '7_5', '7_6','7_3', '7_4']
col8 = ['8_1','8_2', '8_5', '8_6','8_3', '8_4']
result7 = [];result8 = []
for emd in result.emd_nm:
    print(emd)
    if emd in no_prophet:
        v = np.mean(result.loc[result.emd_nm ==emd,col7].mean())
        vv = np.mean(result.loc[result.emd_nm ==emd,col8].mean())
        result7.append(np.round(v,1))
        result8.append(np.round(vv,1))
    else: 
        col7p = col7 + ['prophet_7']
        col8p = col8 + ['prophet_8']
        v = np.mean(result.loc[result.emd_nm ==emd,col7p].mean())
        vv = np.mean(result.loc[result.emd_nm ==emd,col8p].mean())
        result7.append(np.round(v,1))
        result8.append(np.round(vv,1))
result = pd.concat([result.emd_nm,pd.DataFrame(result7),pd.DataFrame(result8)],axis=1)
result.columns = ['행정동명','7월 배출량(g)','8월 배출량(g)']

건입동
구좌읍
남원읍
노형동
대륜동
대정읍
대천동
도두동
동홍동
봉개동
삼도1동
삼도2동
삼양동
서홍동
성산읍
송산동
아라동
안덕면
애월읍
연동
영천동
예래동
오라동
외도동
용담1동
용담2동
이도1동
이도2동
이호동
일도1동
일도2동
정방동
조천읍
중문동
중앙동
천지동
표선면
한경면
한림읍
화북동
효돈동


In [501]:
result.to_csv("final.csv")

In [98]:
emd_ls = emd_ls_1 + emd_ls_2 + emd_ls_5 + emd_ls_6
print(set(data.emd_nm) - set(emd_ls))
print(set(emd_ls) - set(data.emd_nm))

{'대천동', '중문동', '남원읍', '영천동', '예래동', '대륜동', '일도2동', '구좌읍', '중앙동', '화북동', '일도1동', '정방동', '삼도2동', '동홍동', '서홍동', '송산동', '천지동'}
set()
